In [10]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
#locate files from computer 


In [ ]:

# locate and load file to Read 
df = pd.read_csv('trippub.csv')
# create a person ID for the dataframe, this is to identify specific persons
df['Person'] = df['HOUSEID'].map(str) + '-' + df['PERSONID'].map(str)
df=df.loc[ :, ['HOUSEID','PERSONID','Person','HHFAMINC','HHSTATE','HHSIZE','HHVEHCNT','LIF_CYC','HH_RACE','R_SEX','R_AGE','TRAVDAY','DWELTIME','TRPMILES','TRPTRANS','TRVLCMIN','URBRUR','WHYFROM','WHYTO','WTTRDFIN']]
# Delete the unknown and remove unknown dwelltime. 
df1 = df.replace({'DWELTIME':{-9:1000}})
df1

In [ ]:
#Definitions of Tour, Stop and primary destination.
#Here we merge the two columns to understand the travel pattern. The definitions are based on the time spent at the location and the type pf location.

df1["Dailypattern"] = df1["Primary stop"].astype(str) + df1["WHYTO"]

# The logic here is that a 'stop' is a short stop at a location less than 2 hours, a 'Tour' is a longer stop often over night, also it is where most trips are assumed to start from. 
# A 'p_destination' is a primary location an individual commutes to. 
df1=df1.replace(to_replace ="short Stop at Home", value ="Tour") 
df1=df1.replace(to_replace ="short Stop at Stop", value ="Stop") 
df1=df1.replace(to_replace ="PrimaryStop", value ="P_Destination") 
df1=df1.replace(to_replace ="PrimaryHome", value ="Tour") 
df1

In [ ]:
# define if is Tour, Stop or P Destination
# run individual codes for the differents patterns
df2=df1.loc[df1.Dailypattern=='Tour', :]
#df2=df1.loc[df1.Dailypattern=='Stop', :]
#df2=df1.loc[df1.Dailypattern=='P_Destination', :]
df2

In [ ]:
#count the numbers of tours, stops and p.destination, 
# change the dependent variable to count the tours, stops and p.d 
#This will provide different columns 

df2['Tour'] = df2.groupby('Person')['Dailypattern'].transform('count')
#df2['Stop'] = df2.groupby('Person')['Dailypattern'].transform('count')
#df2['P_Destination'] = df2.groupby('Person')['Dailypattern'].transform('count')
df2

In [ ]:
# Here you can select between person level tour or house level tour. 

tour = df2.groupby(['Person']).mean()
#tour = df1.groupby(['HOUSEID']).mean()
tour

In [ ]:
#tour.to_csv('tourfile.csv')
#tour.to_csv('stopfile.csv')
#tour.to_csv('pdestinationfile.csv')

# import the person or household file to merge, for household level tour or person level tour

In [ ]:
#The point of merging is to use the person or household sample weights. 
#you can break the file at this poin if its too large. 

In [ ]:
person = pd.read_csv('Perpub.csv')
tour = pd.read_csv('tourfile.csv')
stop = pd.read_csv('stopfile.csv')
pdestination = pd.read_csv('pdestinationfile.csv')

In [ ]:
#Merge trip and person level file and select the person level or household sample weights
persontour = tour.join(person, lsuffix="_left", rsuffix="_right")
persontour

In [ ]:
#Merge trip and person level file and select the person level or household sample weights
persontourstop = stop.join(persontour, lsuffix="_left", rsuffix="_right")
persontourstop

In [ ]:
#Merge trip and person level file and select the person level or household sample weights
merged = pdestination.join(persontourstop, lsuffix="_left", rsuffix="_right")
merged